In [1]:
import os 
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Sanjeev_Moparthi/WineQuality_prediction_dataScience_project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Sanjeev_Moparthi"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "333be33f47e4aaaa15c23950a1655b9c3000fbbe"

In [2]:
import os
%pwd

'c:\\WineQuality_prediction_dataScience_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\WineQuality_prediction_dataScience_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: Path
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories,save_json



In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params= self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/Sanjeev_Moparthi/WineQuality_prediction_dataScience_project.mlflow"

        )
        return model_evaluation_config




In [8]:
import os 
import pandas as pd 
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [13]:
# class ModelEvaluation:
#     def __init__(self,config: ModelEvaluationConfig):
#         self.config = config

#     def eval_metrics(self,actual,pred):
#         rmse = np.sqrt(mean_squared_error(actual,pred))
#         mae = mean_absolute_error(actual,pred)
#         r2 = r2_score(actual,pred)
#         return rmse,mae,r2


#     def log_into_mlflow(self):
#         test_data = pd.read_csv(self.config.test_data_path)
#         model = joblib.load(self.config.model_path)

#         test_x = test_data.drop([self.config.target_column],axis = 1)
#         test_y = test_data[[self.config.target_column]]

#         mlflow.set_registry_uri(self.config.mlflow_uri)
#         tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

#         with mlflow.start_run():
#             predicted_qualities = model.predict(test_x)

#             (rmse,mae,r2) = self.eval_metrics(test_y,predicted_qualities)

#             # Saving metrics as local 
#             scores = {"rmse": rmse,"mae": mae,"r2": r2}
#             save_json(path = Path(self.config.metric_file_name),data = scores)

#             mlflow.log_params(self.config.all_params)

#             mlflow.log_metric("rmse",rmse)
#             mlflow.log_metric("r2", r2)
#             mlflow.log_metric("mae",mae)

#             # Model registry does not work with file store 
#             if  tracking_url_type_store != "file":

#                 # Register the model 
#                 # There are other ways to use the model registry,which depends upon on the use cases,
#                 # pLEASE REFER TO THE DOC FOR MORE INFORMATION
#                 mlflow.sklearn.log_model(model,"model") #registered_model_name = "ElasticnetModel"
#             else:
#                 mlflow.sklearn.log_model(model,"model")



class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        netloc = urlparse(mlflow.get_tracking_uri()).netloc

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # 🛡️ Safe model logging: Only if not on DagsHub
            if "dagshub.com" not in netloc and tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            elif tracking_url_type_store == "file":
                mlflow.sklearn.log_model(model, "model")
            else:
                print("⚠️ Skipping model logging due to unsupported MLflow endpoint (e.g., DagsHub).")




In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config = model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-07-20 15:28:00,307: INFO: common: YAML file loaded successfully: config\config.yaml]
[2025-07-20 15:28:00,311: INFO: common: YAML file loaded successfully: params.yaml]
[2025-07-20 15:28:00,315: INFO: common: YAML file loaded successfully: schema.yaml]
[2025-07-20 15:28:00,318: INFO: common: created directory at: artifacts]
[2025-07-20 15:28:00,320: INFO: common: created directory at: artifacts/model_evaluation]
[2025-07-20 15:28:00,800: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
⚠️ Skipping model logging due to unsupported MLflow endpoint (e.g., DagsHub).
🏃 View run trusting-cub-989 at: https://dagshub.com/Sanjeev_Moparthi/WineQuality_prediction_dataScience_project.mlflow/#/experiments/0/runs/45859533593945028cbb9239455b9c9c
🧪 View experiment at: https://dagshub.com/Sanjeev_Moparthi/WineQuality_prediction_dataScience_project.mlflow/#/experiments/0
